In [31]:
# import csv and compress with pca into 2 dimensions
# then plot the data
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import sys

# read in the data
df = pd.read_excel('q-guide-fall.xlsx')
# set the first row to be the column names
# df.columns = df.iloc[0]

In [32]:
df.head()

,web-scraper-order,web-scraper-start-url,link,link-href,coursename,Raters,Students,Count,Excellent,Very Good,...,Unsatisfactory.1,Instructor Mean,Statistics,Value,Options,Score,Count.2,Percentage,Options.1,Count.3
0,1669652682-1,https://qreports.fas.harvard.edu/home/courses?...,WOMGEN 1421-Medical Management of the Female B...,https://harvard.bluera.com/harvard/rpv-eng.asp...,WOMGEN 1421,Responded,14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1669652682-2,https://qreports.fas.harvard.edu/home/courses?...,WOMGEN 1421-Medical Management of the Female B...,https://harvard.bluera.com/harvard/rpv-eng.asp...,WOMGEN 1421,Invited,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1669652682-3,https://qreports.fas.harvard.edu/home/courses?...,WOMGEN 1421-Medical Management of the Female B...,https://harvard.bluera.com/harvard/rpv-eng.asp...,WOMGEN 1421,Response Ratio,82%,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1669652682-4,https://qreports.fas.harvard.edu/home/courses?...,WOMGEN 1421-Medical Management of the Female B...,https://harvard.bluera.com/harvard/rpv-eng.asp...,WOMGEN 1421,NaN,NaN,14.0,71%,29%,...,0%,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,14.0
4,1669652682-5,https://qreports.fas.harvard.edu/home/courses?...,WOMGEN 1421-Medical Management of the Female B...,https://harvard.bluera.com/harvard/rpv-eng.asp...,WOMGEN 1421,NaN,NaN,14.0,86%,14%,...,0%,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,14.0


In [33]:
# drop column web-scraper-order
df = df.drop(['web-scraper-order', 'web-scraper-start-url'], axis=1)

In [34]:
# drop the first row
# df = df.drop(df.index[0])
# df.columns = df.columns.str.strip()
# replace all NRP with 0
df = df.replace('NRP', 0)

In [39]:
formatted_df = pd.DataFrame(columns=[
  'name',
  'class_size',
  'rsp_ratio',
  'course_overall',
  'materials',
  'assignments',
  'feedback',
  'section',
  'instructor',
  'lectures',
  'accessible',
  'enthusiasm',
  'discussion',
  'feedback',
  'timely', 
  'time_mean',
  'time_std',
  'time_median',
  'recommend',
])
def formatting_fnc2(group):
  # some few courses have no instructor ratings; we exclude them
  if group['Instructor Mean'].dropna().shape[0] == 0:
    return pd.DataFrame()
  return_df = pd.DataFrame(columns=['reason'], index=[0])
  return_df['name'] = group['name'].iloc[0]
  # select from group row where raters = 'Response Ratio'
  class_size = float(group[group['Raters'] == 'Invited']['Students'].iloc[0])
  return_df['class_size'] = class_size ** -1 if class_size > 0 else 0
  return_df['rsp_ratio'] = float(group[group['Raters'] == 'Response Ratio']['Students'].iloc[0][0:-1])/100
  # General feedback
  return_df['course_overall'] = float(group['Course Mean'].dropna().iloc[0]) / 5
  return_df['materials'] = float(group['Course Mean'].dropna().iloc[1]) / 5
  return_df['assignments'] = float(group['Course Mean'].dropna().iloc[2]) / 5
  return_df['feedback'] = float(group['Course Mean'].dropna().iloc[3]) / 5
  return_df['section'] = float(group['Course Mean'].dropna().iloc[4]) / 5
  # Instructor
  return_df['instructor'] = float(group['Instructor Mean'].dropna().iloc[0]) / 5
  return_df['lectures'] = float(group['Instructor Mean'].dropna().iloc[1]) / 5
  return_df['accessible'] = float(group['Instructor Mean'].dropna().iloc[2]) / 5
  return_df['enthusiasm'] = float(group['Instructor Mean'].dropna().iloc[3]) / 5
  return_df['discussion'] = float(group['Instructor Mean'].dropna().iloc[4]) / 5
  return_df['feedback'] = float(group['Instructor Mean'].dropna().iloc[5]) / 5
  return_df['timely'] = float(group['Instructor Mean'].dropna().iloc[6]) / 5
  # time
  # normalized such that statistics are centered around 0.5
  time_mean = float(group[group['Statistics'] == 'Mean']['Value'].iloc[0]) / 11.1
  return_df['time_mean'] = time_mean
  time_std = float(group[group['Statistics'] == 'Standard Deviation']['Value'].iloc[0]) / 4.74
  return_df['time_std'] = time_std
  time_med = float(group[group['Statistics'] == 'Median']['Value'].iloc[0]) / 11
  return_df['time_median'] = time_med
  recommend_rows = group[df['Options'].str.contains('Recommend', na=False)]
  return_df.at[0, 'recommend'] = recommend_rows['Percentage'].str.strip('%').astype('float').div(100).mean() / 5
  return_df['elective'] = group[group['Options'] == 'Elective']['Count'].iloc[0]/class_size
  return_df['concentration'] = group[group['Options'] == 'Concentration or Department Requirement']['Count'].iloc[0]/class_size
  return_df['secondary'] = group[group['Options'] == 'Secondary Field or Language Citation Requirement']['Count'].iloc[0]/class_size
  return return_df

In [40]:
df['name'] = df['coursename']
grouped = df.groupby('coursename')

In [41]:
# group by course name column
# disable output
formatted_df = grouped.apply(formatting_fnc2)

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4px4ghs00000gn/T/ipykernel_69080/1257539100.py:54: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/pw/wpnw6hs16b35mpds4p

In [42]:
# drop name columns
# formatted_df = formatted_df.drop(['name'], axis=1)
# convert index to numbers
formatted_df = formatted_df.reset_index(drop=True)
# drop all zero rows
formatted_df = formatted_df[(formatted_df.T != 0).any()]

In [43]:
# drop columns reason
formatted_df = formatted_df.drop(['reason'], axis=1)

In [44]:
# display all rows in formatted_df containing nan
formatted_df[formatted_df.isna().any(axis=1)]

,name,class_size,rsp_ratio,course_overall,materials,assignments,feedback,section,instructor,lectures,...,enthusiasm,discussion,timely,time_mean,time_std,time_median,recommend,elective,concentration,secondary
221,ECON 2912,0.047619,0.38,1.000,0.000,0.000,0.8,0.000,1.00,0.000,...,0.800,0.0,0.0,0.18018,NaN,0.181818,0.04,0.047619,0.000000,0.0
222,ECON 3011,0.058824,0.35,1.000,1.000,0.000,1.0,0.000,1.00,1.000,...,1.000,1.0,0.0,0.27027,NaN,0.272727,0.04,0.058824,0.117647,0.0
223,ECON 3012,0.023810,0.24,1.000,0.000,0.000,0.0,0.000,1.00,1.000,...,1.000,1.0,0.0,0.18018,NaN,0.181818,0.04,0.023810,0.000000,0.0
704,OEB 399,0.100000,0.60,0.766,0.866,0.866,1.0,0.866,0.88,0.866,...,0.934,0.8,1.0,0.18018,NaN,0.181818,0.04,0.000000,0.600000,0.0
867,STAT 300HFRA,0.076923,0.54,1.000,1.000,1.000,1.0,1.000,1.00,1.000,...,1.000,1.0,1.0,0.45045,NaN,0.454545,0.04,0.000000,0.230769,0.0


In [69]:
# fill all cells containing nan with 0
formatted_df = formatted_df.fillna(0)
# math_courses = formatted_df[formatted_df['name'].str.contains('MATH')]
# copy math_courses
# math_courses_copy = math_courses.copy()

In [70]:
pca = PCA(n_components=2)
data = pca.fit_transform(formatted_df.drop(['name'], axis=1))

In [74]:
# do PCA on formatted_df
# merge data with formatted_df['name']
data = pd.DataFrame(data, columns=['x', 'y'])
# math_courses_copy.index = data.index
data['name'] = formatted_df['name']
# output what percent of variance is explained by each component
pca.explained_variance_ratio_

array([0.3603967 , 0.19802422])

In [75]:
# print the row with name like math 22a
# data[data['name'].str.contains('MATH 22A')]
data.head()

,x,y,name
0,-0.410641,0.598437,AFRAMER 11
1,-0.341913,-0.036146,AFRAMER 116
2,-0.263277,0.367801,AFRAMER 134X
3,-0.401495,-0.084959,AFRAMER 183Y
4,-0.474046,0.049253,AFRAMER 197


In [78]:
# plot the data in plotly (interactive)
# labelling by name
fig = go.Figure(data=go.Scatter(
    x=data['x'],
    y=data['y'],
    mode='markers',
    text=data['name'],
    marker=dict(
        size=3,
        color=formatted_df['time_median'] * 5.5, # set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))
fig.show()